MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I submit jobs to my high performance computer to perform the preprocessing, confound_correction, and analysis steps as implemented in RABIES. 
This section relies on the fact that my environment allows for parallel job submission using the `qsub` command. 



In [7]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/scratch/m/mchakrav/gjoanes/multirat/test'      # location of the bids folder and results
df_path='../assets/table/multiRat_rest_20220414.tsv'    # meta-data table

scratch_folder='/scratch/m/mchakrav/gjoanes/multirat/test/scratch' # this is where preprocessed data will be temporarily stored
rabies_img='/project/m/mchakrav/gjoanes/rabies-0.3.5.simg' # location of the RABIES image

`scp -r job_sub-010*  gjoanes@niagara.computecanada.ca://scratch/m/mchakrav/gjoanes/multirat/test/qsub/queued ` 
for safe keeping. 

In [ ]:
# init bunch of export folders
! mkdir -p $analysis_folder/qsub/queued

! mkdir -p $analysis_folder/export/qa/epi2anat
! mkdir -p $analysis_folder/export/qa/anat2template
! mkdir -p $analysis_folder/export/qa/template2std
! mkdir -p $analysis_folder/export/qa/temporal
! mkdir -p $analysis_folder/export/snr/atlas
! mkdir -p $analysis_folder/export/snr/tsnr
! mkdir -p $analysis_folder/export/motion/confound
! mkdir -p $analysis_folder/export/motion/FD
! mkdir -p $analysis_folder/export/log/preprocess
! mkdir -p $analysis_folder/export/log/confound
! mkdir -p $analysis_folder/export/log/analysis
! mkdir -p $analysis_folder/export/stim
! mkdir -p $analysis_folder/export/aromas_scan
! mkdir -p $analysis_folder/export/aromas_qa

# init complete/error logs
! touch $analysis_folder/export/log/error.log
! touch $analysis_folder/export/log/complete.log


In [11]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

In [3]:
from subprocess import call

qsub_call = "qsub -l 'walltime=48:00:00,mem=24gb, procs=1' "  # define how much time/memory/core to allocate per job


module_args = "module load gcc/8.3.0; module load openblas/0.3.7; module load fsl/6.0.4"  #the freesurfer and ants modules tricks as specific to my system due to some env contamination bugs 
fsl_load_args='module load fsl'
fsl_unload_args='module unload fsl'

template_args = "template=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Template.nii; mask=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_Brain_Mask.nii; wm=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_WM_bin.nii.gz; csf=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Anatomical_Imaging/SIGMA_Rat_Anatomical_InVivo_Template/SIGMA_InVivo_CSF_bin.nii.gz; atlas=/template/SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_rs.nii; roi=/template/roi/"


rabies_runmode_args = "-p MultiProc "
rabies_preprocess_args = "preprocess /rabies_in /rabies_out \
 --anat_template ${template} \
--brain_mask ${mask} \
--WM_mask ${wm} \
--CSF_mask ${csf} \
--vascular_mask ${csf} \
--labels ${atlas} \
--TR ${TR} \
--fast_commonspace \
--commonspace_resampling 0.3x0.3x0.3 \
--anat_inho_cor_method N4_reg \
--bold_inho_cor_method N4_reg \
--anat_autobox \
--coreg_script Rigid "

rabies_confound_args = "confound_correction /rabies_out /confound_out \
--read_datasink \
--TR ${TR} \
--edge_cutoff 0 \
--highpass 0.01 \
--smoothing_filter 0.5 ${param}"

rabies_analysis_args = "analysis /confound_out /analysis_out \
--FC_matrix \
--seed_list /roi_folder/ACA_l.nii.gz /roi_folder/CPu_l.nii.gz /roi_folder/MOp_l.nii.gz /roi_folder/S1bf_l.nii.gz"


In [13]:
#for index in df.iterrows():
for index in range(741,752):

    # get information from dataframe 
    subject_ID="sub-0"+str(df.iloc[index]['rat.sub'])
    session_ID="ses-"+str(df.iloc[index]['rat.ses'])

    TR="TR="+str(df.iloc[index]['func.TR'])
    isrest=df.iloc[index]['exp.type']=='resting-state'

    # set the base folder structure and init folders

    origbids_args="orig_folder="+os.path.join(analysis_folder, "bids", subject_ID,session_ID)
    template_folder_args="template_folder="+os.path.join(analysis_folder, "template")

    folderbids_args="bids_folder="+os.path.join(scratch_folder, "bids", subject_ID+"_"+session_ID)
    folderprep_args="preprocess_folder="+os.path.join(scratch_folder, "preprocess", subject_ID+"_"+session_ID)

    clearfolder_args="rm -rf $bids_folder; rm -rf $preprocess_folder; mkdir -p $bids_folder/"+subject_ID+"; mkdir -p $preprocess_folder"
    cpbids_args='cp -r '+os.path.join(analysis_folder, "bids", subject_ID,session_ID)+" $bids_folder/"+subject_ID+"/"

    # get error message if any error occurred during preprocessing 
    pre_error_args='now=`date`; ! grep -q "Error" ${preprocess_folder}/rabies_preprocess.log || echo $now" "'+analysis_folder+ '/export/log/preprocess/' + subject_ID+'_'+session_ID+'.log >> '+analysis_folder+'export/log/error.log'

    # copy preprocessing outputs of interest to designated folders
    pre_cp1_args='cp ${preprocess_folder}/rabies_preprocess.log ' +analysis_folder+ '/export/log/preprocess/' + subject_ID+'_'+session_ID+'.log'
    pre_cp2_args='cp ${preprocess_folder}/preprocess_QC_report/EPI2Anat/* '+analysis_folder+ '/export/qa/epi2anat/'
    pre_cp3_args='cp ${preprocess_folder}/preprocess_QC_report/temporal_features/* '+analysis_folder+ '/export/qa/temporal/'
    pre_cp4_args='cp ${preprocess_folder}/preprocess_QC_report/commonspace_reg_wf.Native2Atlas/*png '+analysis_folder+ '/export/qa/anat2template/'

    pre_cp5_args='cp ${preprocess_folder}/bold_datasink/commonspace_labels/*/*/*.nii.gz '+analysis_folder+ '/export/snr/atlas/'
    pre_cp6_args='cp ${preprocess_folder}/confounds_datasink/confounds_csv/*/*/*.csv '+analysis_folder+ '/export/motion/confound/'
    pre_cp7_args='cp ${preprocess_folder}/confounds_datasink/FD_csv/*/*/*.csv  '+analysis_folder+ '/export/motion/FD/'


    pre_cp8_args='ls ${preprocess_folder}/bold_datasink/commonspace_labels | while read label; do label_path=$(find ${preprocess_folder}/bold_datasink/commonspace_labels/${label}/*/*.nii.gz); tSNR_path=$(find ${preprocess_folder}/bold_datasink/tSNR_map_preprocess/${label}/*/*.nii.gz); mask_path=$(find ${preprocess_folder}/bold_datasink/commonspace_mask/${label}/*/*.nii.gz); fslmeants -i ${tSNR_path} -m ${mask_path} --label=${label_path} -o '+ analysis_folder+ '/export/snr/tsnr/${label}.txt; done'


    # resting-state post-processing
    rest1_args='denoise_list=("aromas" "GSRs" "WMCSFs" "aromasr" "aromal"); denoise_parameters=("--lowpass 0.1 --run_aroma --aroma_dim 10" "--lowpass 0.1 --conf_list global_signal mot_6" "--lowpass 0.1 --conf_list WM_signal CSF_signal mot_6" "--lowpass 0.1 --run_aroma --aroma_dim 10 --FD_censoring" "--lowpass 0.2 --run_aroma --aroma_dim 10" )'
    rest2_args='for denoise in ${denoise_list[*]}; do id=$(echo ${denoise_list[@]/$denoise//} | cut -d/ -f1 | wc -w | tr -d " "); param=${denoise_parameters[$id]}'
    rest3_args='current_confound='+os.path.join(scratch_folder,'confound')+'/${denoise}/'+subject_ID+'_'+session_ID
    rest4_args='current_analysis='+os.path.join(scratch_folder,'analysis')+'/${denoise}/'+subject_ID+'_'+session_ID
    rest5_args='rm -rf $current_confound; rm -rf $current_analysis; mkdir -p $current_confound; mkdir -p $current_analysis'

    # get error message if any error occurred during confound regression 
    rest_error1_args='now=`date`; ! grep -q "Error" ${current_confound}/rabies_confound_correction.log || echo $now" "'+analysis_folder+'/export/log/confound/' + subject_ID+'_'+session_ID+'_${denoise}.log >> '+analysis_folder+'export/log/error.log'

    # copy aromas confound outputs of interest to designated folders
    rest_cp1_args='cp ${current_confound}/rabies_confound_correction.log ' +analysis_folder+ '/export/log/confound/' + subject_ID+'_'+session_ID+'_${denoise}.log'
    rest_cp2_args='if [ "$denoise" == "aromas" ]; then cp -r ${current_confound}/confound_correction_wf_datasink/cleaned_timeseries/*/*.nii.gz '+analysis_folder+'/export/aromas_scan; ls ${current_confound}/confound_correction_wf_datasink/aroma_out | while read label; do mkdir -p '+analysis_folder+'/export/aromas_qa/${label}; cp -r ${current_confound}/confound_correction_wf_datasink/aroma_out/${label}/aroma_out/melodic.ica/report/* '+analysis_folder+'/export/aromas_qa/${label}; cp -r ${current_confound}/confound_correction_wf_datasink/aroma_out/${label}/aroma_out/classified_motion_ICs.txt '+analysis_folder+'/export/aromas_qa/${label}/classification.txt; done;fi'

    # get error message if any error occurred during analysis 
    rest_error2_args='now=`date`; ! grep -q "Error" ${current_analysis}/rabies_analysis.log || echo $now" "'+analysis_folder+'/export/log/analysis/' + subject_ID+'_'+session_ID+'_${denoise}.log >> '+analysis_folder+'export/log/error.log'


    # copy seed-based maps to designated folders
    rest_cp3_args='cp ${current_analysis}/rabies_analysis.log ' +analysis_folder+ '/export/log/analysis/' + subject_ID+'_'+session_ID+'_${denoise}.log'
    rest_cp4_args='seed_dir='+analysis_folder+'/export/seed/${denoise}; mkdir -p ${seed_dir}; cp -r ${current_analysis}/analysis_wf_datasink/seed_correlation_maps/*/*/* $seed_dir'
    rest_cp5_args='NA_dir='+analysis_folder+'/export/NA/${denoise}; mkdir -p ${NA_dir}; cp -r ${current_analysis}/analysis_wf_datasink/matrix_data_file/*/* $NA_dir'


    rest6_args='done'

    #clean up
    clear_args='rm -rf $bids_folder; rm -rf $preprocess_folder; rm -rf '+os.path.join(scratch_folder,'confound')+' rm -rf '+os.path.join(scratch_folder,'analysis')

    #report completed script to log file. Doesnt mean completed is error free
    complete_args='echo $now" "'+subject_ID+'_'+session_ID+' >> '+analysis_folder+'/export/log/complete.log'

    qsub_job_name=os.path.join('/project/4180000.19/multiRat', 'qsub','queued', 'job_'+subject_ID+'_'+session_ID +'.sh')

    with open (qsub_job_name, 'w') as rsh:
        rsh.writelines("#"+'!/bin/bash' +'\n')
        rsh.writelines("#"+'SBATCH --time=24:00:00' +'\n')
        rsh.writelines("#"+'SBATCH --nodes=1' +'\n')
        rsh.writelines("#"+'SBATCH --account=rrg-mchakrav-ab' +'\n')
        rsh.writelines("#"+'SBATCH --ntasks=40' +'\n')
        rsh.writelines(module_args +'\n')
        rsh.writelines(template_args +'\n')
        rsh.writelines(TR +'\n')
        rsh.writelines(origbids_args +'\n')
        rsh.writelines(template_folder_args +'\n')
        rsh.writelines(folderbids_args +'\n')
        rsh.writelines(folderprep_args +'\n')
        rsh.writelines(clearfolder_args +'\n')
        rsh.writelines(cpbids_args +'\n')
        rsh.writelines('singularity run -B ${template_folder}:/template -B ${bids_folder}:/rabies_in:ro ')
        rsh.writelines('-B ${preprocess_folder}:/rabies_out ')
        rsh.writelines(rabies_img+' ')
        rsh.writelines(rabies_runmode_args+' ')
        rsh.writelines(rabies_preprocess_args +'\n')
        rsh.writelines(pre_error_args +'\n')
        rsh.writelines(pre_cp1_args +'\n')
        rsh.writelines(pre_cp2_args +'\n')
        rsh.writelines(pre_cp3_args +'\n')
        rsh.writelines(pre_cp4_args +'\n')
        rsh.writelines(pre_cp5_args +'\n')
        rsh.writelines(pre_cp6_args +'\n')
        rsh.writelines(pre_cp7_args +'\n')
        rsh.writelines(pre_cp8_args +'\n')
        rsh.writelines('module unload fsl' +'\n')
        # add if statement here for resting state
        rsh.writelines(rest1_args +'\n')
        rsh.writelines(rest2_args +'\n')
        rsh.writelines(rest3_args +'\n')
        rsh.writelines(rest4_args +'\n')
        rsh.writelines(rest5_args +'\n')
        rsh.writelines('singularity run -B ${template_folder}:/template -B ${bids_folder}:/rabies_in:ro ')
        rsh.writelines('-B ${preprocess_folder}:/rabies_out -B ${current_confound}:/confound_out ')
        rsh.writelines(rabies_img+' ')
        rsh.writelines(rabies_confound_args +'\n')
        rsh.writelines(rest_error1_args +'\n')
        rsh.writelines(rest_cp1_args +'\n')
        rsh.writelines(rest_cp2_args +'\n')
        rsh.writelines('singularity run -B ${template_folder}/roi:/roi_folder -B ${template_folder}:/template -B ${bids_folder}:/rabies_in:ro ')
        rsh.writelines('-B ${preprocess_folder}:/rabies_out -B ${current_confound}:/confound_out -B ${current_analysis}:/analysis_out ')
        rsh.writelines(rabies_img+' ')
        rsh.writelines(rabies_analysis_args +'\n')
        rsh.writelines(rest_error2_args +'\n')
        rsh.writelines(rest_cp3_args +'\n')
        rsh.writelines(rest_cp4_args+'\n')
        rsh.writelines(rest_cp5_args+'\n')
        rsh.writelines(rest6_args +'\n')
        #rsh.writelines(clear_args +'\n')
        rsh.writelines(complete_args +'\n')



IndexError: single positional indexer is out-of-bounds